In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns

import janestreet
env = janestreet.make_env() # initialize the environment

#!pip install datatable # Internet is not activated in this competition
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl
import datatable as dt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

INPUT_DIR = '/kaggle/input/jane-street-market-prediction/'

Processing /kaggle/input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl


/kaggle/input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl
/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so


Thanks to following notebooks :  
https://www.kaggle.com/drcapa/jane-street-market-prediction-starter-xgb/output?select=example_sample_submission.csv

# Load data

In [2]:
%%time
# Thanks to his notebook for this fast loading : https://www.kaggle.com/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance
train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
df = train_data_datatable.to_pandas()

# Thanks to this notebook to gain memory usage : https://www.kaggle.com/jorijnsmit/one-liner-to-halve-your-memory-usage
float64_cols = df.select_dtypes(include='float64').columns
mapper = {col_name: np.float32 for col_name in float64_cols}
df = df.astype(mapper)

CPU times: user 46.7 s, sys: 14.5 s, total: 1min 1s
Wall time: 24.2 s


# Make random predictions

Chek mean of positive returns :

In [3]:
df['resp_positive'] = ((df['resp'])>0)*1

In [4]:
action_1_proba = df[df['resp_positive'] == 1].shape[0] / df.shape[0]
print(action_1_proba)

0.504069666022587


In [5]:
np.random.seed(42)

In [6]:
series_random_preds = pd.Series(np.asarray(np.random.rand(df.shape[0]) > (1 - action_1_proba), dtype=int))

In [7]:
series_random_preds.mean()

0.5038575757030669

# Submit predictions

In [8]:
iter_test = env.iter_test() # an iterator which loops over the test set

In [9]:
%%time
cnt_samples = 0

for (test_df, sample_prediction_df) in iter_test:
    cnt_samples += 1
    
    sample_prediction_df.action = series_random_preds[cnt_samples]
    
    env.predict(sample_prediction_df)

CPU times: user 3min 16s, sys: 407 ms, total: 3min 16s
Wall time: 3min 16s


In [10]:
print(f'There are {cnt_samples} samples predicted')

There are 15219 samples predicted
